In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, round

# Inicializar Spark Session con memoria suficiente
spark = SparkSession.builder \
    .appName("AutoInsights_ETL") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("Spark Inicializado:", spark.version)

Spark Inicializado: 3.5.0


In [2]:
# Ruta interna del contenedor (mapeada por Docker)
file_path = "/home/jovyan/work/data/vehicles.csv"

# Leer CSV con header y esquema inferido
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Cachear para velocidad
df.cache()

print(f"Total de registros cargados: {df.count()}")
df.printSchema()

Total de registros cargados: 441802
root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: string (nullable = true)
 |-- year: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- size: string (nullable = true)
 |-- type: string (nullable = true)
 |-- paint_color: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable =

In [3]:
# 1. Eliminar filas con precio irreal (menor a $500 o mayor a $1M)
# 2. Eliminar años futuros o muy viejos (1990 - 2025)
# 3. Eliminar filas sin marca o modelo

df_clean = df.filter(
    (col("price") > 500) & 
    (col("price") < 1000000) & 
    (col("year") > 1990) & 
    (col("year") <= 2025) &
    col("manufacturer").isNotNull() &
    col("model").isNotNull()
)

print(f"Registros después de limpieza: {df_clean.count()}")

Registros después de limpieza: 354813


In [4]:
# Calcular precio promedio por Marca y Año
df_agg = df_clean.groupBy("manufacturer", "year") \
    .agg(
        round(avg("price"), 2).alias("avg_price"),
        count("*").alias("count")
    ) \
    .orderBy("manufacturer", "year")

# Mostrar una muestra
df_agg.show(20)

+------------+-----+---------+-----+
|manufacturer| year|avg_price|count|
+------------+-----+---------+-----+
|        2015| 2014|   2013.0|  164|
|       acura| 1991| 59316.67|    6|
|       acura| 1992|   3200.0|    1|
|       acura| 1993|   1550.0|    3|
|       acura| 1994|   4365.0|    3|
|       acura| 1995|   2600.0|    3|
|       acura| 1996| 10015.44|    9|
|       acura| 1997|  3890.73|   11|
|       acura| 1998|   2500.0|    5|
|       acura| 1999|  3559.63|   19|
|       acura| 2000|  3123.65|   23|
|       acura| 2001|  3529.42|   45|
|       acura| 2002|  3718.86|   49|
|       acura| 2003|  5548.32|   98|
|       acura| 2004|  5863.49|  111|
|       acura| 2005|  5914.15|  171|
|       acura| 2006|  6064.25|  201|
|       acura| 2007|  7486.33|  153|
|       acura| 2008|  7948.79|  165|
|       acura| 2009|  8965.91|  151|
+------------+-----+---------+-----+
only showing top 20 rows

